# <B>Scalable Data Mining </B>
## Assignment : 2
### Name :  Altaf Ahmad 
### Roll no : 18MA20005

Training and Testing Convolutional Neural Networks for image classification on CIFAR10 dataset using PyTorch Module

In [ ]:
!pip install wandb

     |████████████████████████████████| 1.7 MB 5.4 MB/s 
     |████████████████████████████████| 97 kB 6.6 MB/s 
     |████████████████████████████████| 133 kB 40.8 MB/s 
     |████████████████████████████████| 180 kB 46.1 MB/s 
     |████████████████████████████████| 63 kB 1.8 MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-py3-none-any.whl size=6502 sha256=954b2cf6cef6fd74ffb71fc0e506f76eab573a4fa466866d3d7c613131edc027
  Stored in directory: /root/.cache/pip/wheels/50/ca/fa/8fca8d246e64f19488d07567547ddec8eb084e8c0d7a59226a
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8807 sha256=8758194f5c953b92e5feed18224791ec32e39cff11e25f170d77fb7dc576ec24
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built subprocess32 pathtools


In [ ]:
from __future__ import print_function
import argparse
import random
import numpy as np
import os
import sys
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import torchvision
from torchvision import datasets, transforms
import wandb

In [ ]:
wandb.login()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

1. <B>Data Load</b>: Using the file main.py given in the above link to load the data and carry on
further experiments.

In [ ]:
# Data
print('Data transformation')
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

trainset = torchvision.datasets.CIFAR10(
    root='./data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=128, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(
    root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=100, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck')

Data transformation


  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


2. Model: Use the pretrained Resnet18 model to train your Convolutional Neural Network in
the following ways. <br> <b>
a) Train all the layers <br>
b) Freeze the other layers and finetune only the last layer.

In [ ]:
#Model
print('Model creation')

net1_SGD = torchvision.models.resnet18(pretrained = True)
net1_Adam = torchvision.models.resnet18(pretrained = True)
net2_SGD = torchvision.models.resnet18(pretrained = True)
net2_Adam = torchvision.models.resnet18(pretrained = True)

for param in net2_SGD.parameters():
    param.requires_grad = False
    
for param in net2_Adam.parameters():
    param.requires_grad = False

num_features = net1_SGD.fc.in_features
net1_SGD.fc = nn.Linear(num_features,10)
net1_Adam.fc = nn.Linear(num_features,10)
net2_SGD.fc = nn.Linear(num_features,10)
net2_Adam.fc = nn.Linear(num_features,10)
#### net = Invoke pretrained ResNet18 model #######

net1_SGD = net1_SGD.to(device)
net1_Adam = net1_Adam.to(device)
net2_SGD = net2_SGD.to(device)
net2_Adam = net2_Adam.to(device)

Model creation


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

3. Training Module: Implement a mini-batch SGD using main.py to train the CNN in both the
ways described above. Use the following configurations while training: <br>
● Use SGD optimizer with learning rate = 0.001, momentum = 0.9 and cross-entropy as
the loss function.<br>
● Use Adam optimizer with learning rate = 0.01 and cross-entropy as the loss function.
You can use early stopping too if loss converges beforehand.

In [ ]:
optimizer1_SGD = optim.SGD(net1_SGD.parameters(),lr = 0.001,momentum=0.9)
optimizer1_Adam = optim.Adam(net1_Adam.parameters(),lr = 0.01)
optimizer2_SGD = optim.SGD(net2_SGD.fc.parameters(),lr = 0.001,momentum=0.9)
optimizer2_Adam = optim.Adam(net2_Adam.fc.parameters(),lr = 0.01)

In [ ]:
criterion = nn.CrossEntropyLoss()

In [ ]:
# Training
def train(model , optimizer):
    print('\nEpoch: %d' % epoch)
    model.train()
    batch_loss = 0.0            
    correct = 0      
    
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        # Write your code here
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs,targets)
        loss.backward()
        optimizer.step()
        batch_loss += loss.item()
        _,pred = torch.max(outputs,dim = 1)
        correct += torch.sum(pred == targets).item()
    loss = batch_loss/len(trainloader.dataset)
    accuracy = 100*correct/len(trainloader.dataset)
    print("Training Loss : ", loss, ", and Accuracy = ", accuracy)
    return (loss, accuracy)

In [ ]:
def test(model,early_stop,model_name,optim_name,no_improve = 10):
    global best_acc
    global best_loss
    model.eval()

    batch_loss = 0            
    correct_t = 0             
    prediction_samples = []   
    predictions = []          
    targetlist = []  

    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            batch_loss += criterion(outputs, targets).item()
            _,pred_t = torch.max(outputs, dim = 1)
            correct_t += torch.sum(pred_t == targets).item()
            predictions.extend(pred_t)
            targetlist.extend(targets)
            prediction_samples.append(wandb.Image(inputs[0],caption = "Prediction: {} Truth: {}".format(pred_t[0].item(),targets[0])))
        loss = batch_loss/len(testloader.dataset)
        curr_acc = 100*correct_t/len(testloader.dataset)
        print("Testing Loss : ", loss, " and Test Accuracy = ", curr_acc)

        # Save checkpoint for the model which yields best accuracy
        if loss < best_loss:
          predictions = [x.item() for x in predictions]
          targetlist = [x.item() for x in targetlist]
            
          early_stop[1] = 0     
          best_loss = loss   
          torch.save({
          'model_state_dict': model.state_dict(),
          'Testing Loss': loss,
          'Testing Accuracy': curr_acc,
          'Predictions': predictions,
          'Target List': targetlist
          },'resnet18_'+ model_name + '_' + optim_name + '.pth')
        else :
          early_stop[1] += 1
        if early_stop[1] > no_improve:
          early_stop[0] = True
        return (loss, curr_acc, prediction_samples)

First, (a) Training all the layers without freezing, and saving the best model.

In [ ]:
run = wandb.init(name = "1a",project = "Asg1",resume = True)
wandb.watch(net1_SGD, log = "all")
config = wandb.config
config.batch_size = 128
config.test_batch_size = 100
config.lr = 0.001
config.momentum = 0.9

early_stop = [False,0]

global best_loss
best_loss = float('inf')

for epoch in range(200):
  # print(epoch)
  train_loss,train_acc = train(net1_SGD,optimizer1_SGD)
  test_loss,test_acc,samples = test(net1_SGD,early_stop,"Unfreezed","SGD")
  wandb.log({
      "Training Loss":train_loss,
      "Training Accuracy":train_acc,
      "Testing Loss":test_loss,
      "Testing Accuracy": test_acc,
      "Prediction Samples":samples
  })

  if early_stop[0]:
    break


wandb: Currently logged in as: altafahmad (use `wandb login --relogin` to force relogin)



Epoch: 0


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Training Loss :  0.010347825261354446 , and Accuracy =  53.584
Testing Loss :  0.00981736345887184  and Test Accuracy =  65.99

Epoch: 1
Training Loss :  0.007047839354276657 , and Accuracy =  68.406
Testing Loss :  0.008007880449295044  and Test Accuracy =  71.99

Epoch: 2
Training Loss :  0.006064100517034531 , and Accuracy =  72.92
Testing Loss :  0.007125392001867294  and Test Accuracy =  75.41

Epoch: 3
Training Loss :  0.005439881364107132 , and Accuracy =  75.788
Testing Loss :  0.006603970611095428  and Test Accuracy =  76.92

Epoch: 4
Training Loss :  0.005032979090809822 , and Accuracy =  77.606
Testing Loss :  0.006257562786340713  and Test Accuracy =  78.3

Epoch: 5
Training Loss :  0.00470113401055336 , and Accuracy =  79.098
Testing Loss :  0.005985084229707718  and Test Accuracy =  79.41

Epoch: 6
Training Loss :  0.0044409771150350575 , and Accuracy =  80.098
Testing Loss :  0.005817702710628509  and Test Accuracy =  79.84

Epoch: 7
Training Loss :  0.004228492838144303

ValueError: ignored

In [ ]:
resnet18_Unfreezed_SGD = wandb.Artifact('resnet18_Unfreezed_SGD', type='model')
resnet18_Unfreezed_SGD.add_file('resnet18_Unfreezed_SGD.pth')
run.log_artifact(resnet18_Unfreezed_SGD)
best_resnet18_Unfreezed_SGD = torch.load('resnet18_Unfreezed_SGD.pth')

wandb.log({
    "resnet18_Unfrozen_SGD_Confusion_Matrix": wandb.plot.confusion_matrix(
    preds = best_resnet18_Unfreezed_SGD['Predictions'],
    y_true = best_resnet18_Unfreezed_SGD['Target List'],
    class_names = classes) 
})
config.best_test_accuracy = best_resnet18_Unfreezed_SGD['Testing Accuracy']
wandb.finish()

print("Best Model accuracy for Resnet-18 with SDM Optimization" , best_resnet18_Unfreezed_SGD['Testing Accuracy'])

Testing Accuracy,83.78
Testing Loss,0.00556
Training Accuracy,91.772
Training Loss,0.00179


Testing Accuracy,▁▃▅▅▆▆▆▇▇▇▇▇▇▇▇▇██████████████
Testing Loss,█▅▄▃▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂
Training Accuracy,▁▄▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇████████
Training Loss,█▅▄▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁


Best Model accuracy for Resnet-18 with SDM Optimization 83.07


(b) Training all the layers without freezing, and saving the best model. Using Adam this time

In [ ]:
run = wandb.init(name = "1b",project = "Asg1")
wandb.watch(net1_SGD, log = "all")
config = wandb.config
config.batch_size = 128
config.test_batch_size = 100
config.lr = 0.01


early_stop = [False,0]

global best_loss
best_loss = float('inf')

for epoch in range(200):
  # print(epoch)
  train_loss,train_acc = train(net1_Adam,optimizer1_Adam)
  test_loss,test_acc,samples = test(net1_Adam,early_stop,"Unfreezed","Adam")
  wandb.log({
      "Training Loss":train_loss,
      "Training Accuracy":train_acc,
      "Testing Loss":test_loss,
      "Testing Accuracy": test_acc,
      "Prediction Samples":samples
  })

  if early_stop[0]:
    break



Epoch: 0
Training Loss :  0.017420527844429014 , and Accuracy =  20.69
Testing Loss :  0.018313030087947844  and Test Accuracy =  33.1

Epoch: 1
Training Loss :  0.013091442801952362 , and Accuracy =  38.182
Testing Loss :  0.015738428592681885  and Test Accuracy =  44.12

Epoch: 2
Training Loss :  0.011405647790431976 , and Accuracy =  46.658
Testing Loss :  0.013502161979675293  and Test Accuracy =  51.56

Epoch: 3
Training Loss :  0.009872796972990035 , and Accuracy =  54.378
Testing Loss :  0.012820497477054596  and Test Accuracy =  56.01

Epoch: 4
Training Loss :  0.009000726737976075 , and Accuracy =  59.07
Testing Loss :  0.009946665543317796  and Test Accuracy =  65.13

Epoch: 5
Training Loss :  0.00803940614104271 , and Accuracy =  63.398
Testing Loss :  0.009429131251573563  and Test Accuracy =  67.01

Epoch: 6
Training Loss :  0.007528547135591507 , and Accuracy =  66.04
Testing Loss :  0.009522942471504212  and Test Accuracy =  66.45

Epoch: 7
Training Loss :  0.0070999945

In [ ]:
resnet18_Unfreezed_Adam = wandb.Artifact('resnet18_Unfreezed_Adam', type='model')
resnet18_Unfreezed_Adam.add_file('resnet18_Unfreezed_Adam.pth')
run.log_artifact(resnet18_Unfreezed_Adam)
best_resnet18_Unfreezed_Adam = torch.load('resnet18_Unfreezed_Adam.pth')

wandb.log({
    "resnet18_Unfrozen_Adam_Confusion_Matrix": wandb.plot.confusion_matrix(
    preds = best_resnet18_Unfreezed_SGD['Predictions'],
    y_true = best_resnet18_Unfreezed_SGD['Target List'],
    class_names = classes) 
})
config.best_test_accuracy = best_resnet18_Unfreezed_SGD['Testing Accuracy']
wandb.finish()

print("Best Model accuracy for Resnet-18 with Adam Optimization" , best_resnet18_Unfreezed_SGD['Testing Accuracy'])

Testing Accuracy,83.45
Testing Loss,0.00539
Training Accuracy,88.238
Training Loss,0.00264


Testing Accuracy,▁▃▄▄▅▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇█▆██████████████████
Testing Loss,█▇▅▅▄▃▃▃▃▂▃▂▂▃▂▂▁▂▂▂▁▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Training Accuracy,▁▃▄▄▅▅▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇█████████████████
Training Loss,█▆▅▄▄▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁


Best Model accuracy for Resnet-18 with Adam Optimization 83.07


2 (a) Freezing the other layers and fine tuning only the last layer using SGD

In [ ]:
run = wandb.init(name = "2a",project = "Asg1")
wandb.watch(net2_SGD, log = "all")

config = wandb.config
config.batch_size = 128
config.test_batch_size = 100
config.lr = 0.001
config.momentum = 0.9

early_stop = [False,0]

global best_loss
best_loss = float('inf')

for epoch in range(200):
    
    train_loss,train_acc = train(net2_SGD,optimizer2_SGD)
    test_loss,test_acc,samples = test(net2_SGD,early_stop,"Freezed","SGD")
    
    wandb.log({
                "Training Loss":train_loss,
                "Training Accuracy":train_acc,
                "Testing Loss":test_loss,
                "Testing Accuracy": test_acc,
                "Prediction Samples":samples
                })
    
    if early_stop[0] :
        break
        



Epoch: 0
Training Loss :  0.015182906448841096 , and Accuracy =  30.958
Testing Loss :  0.018125318920612334  and Test Accuracy =  36.87

Epoch: 1
Training Loss :  0.013665075402259827 , and Accuracy =  39.014
Testing Loss :  0.017483640253543854  and Test Accuracy =  39.48

Epoch: 2
Training Loss :  0.013357196466922761 , and Accuracy =  40.066
Testing Loss :  0.017252812278270722  and Test Accuracy =  40.1

Epoch: 3
Training Loss :  0.013216595256328583 , and Accuracy =  40.678
Testing Loss :  0.017247617387771606  and Test Accuracy =  39.94

Epoch: 4
Training Loss :  0.01312051253080368 , and Accuracy =  41.424
Testing Loss :  0.017246657812595368  and Test Accuracy =  40.13

Epoch: 5
Training Loss :  0.01305764358997345 , and Accuracy =  41.69
Testing Loss :  0.016882874262332915  and Test Accuracy =  41.22

Epoch: 6
Training Loss :  0.013039245822429657 , and Accuracy =  41.878
Testing Loss :  0.016879298317432404  and Test Accuracy =  41.97

Epoch: 7
Training Loss :  0.012989199

In [ ]:
resnet18_Freezed_SGD = wandb.Artifact('resnet18_Freezed_SGD', type='model')
resnet18_Freezed_SGD.add_file('resnet18_Freezed_SGD.pth')
run.log_artifact(resnet18_Freezed_SGD)

best_resnet18_Freezed_SGD = torch.load('resnet18_Freezed_SGD.pth')

wandb.log({
    "resnet18_Freezed_SGD_Confusion_Matrix": wandb.plot.confusion_matrix(
    preds = best_resnet18_Freezed_SGD['Predictions'],
    y_true = best_resnet18_Freezed_SGD['Target List'],
    class_names = classes) 
})

config.best_test_accuracy = best_resnet18_Freezed_SGD['Testing Accuracy']
print("Best Model accuracy for Resnet-18 Frozen with SDM Optimization" , best_resnet18_Freezed_SGD['Testing Accuracy'])

Best Model accuracy for Resnet-18 Frozen with SDM Optimization 41.88


2 (b) Freezing the other layers and fine tuning only the last layer using Adam

In [ ]:
run = wandb.init(name = "2b",project = "Asg1")
wandb.watch(net2_Adam, log = "all")

config = wandb.config
config.batch_size = 128
config.test_batch_size = 100
config.lr = 0.01

early_stop = [False,0]

global best_loss
best_loss = float('inf')

for epoch in range(200):
    train_loss,train_acc = train(net2_Adam,optimizer2_Adam)
    test_loss,test_acc,samples = test(net2_Adam,early_stop,"Freezed","Adam")
    
    wandb.log({
                "Training Loss":train_loss,
                "Training Accuracy":train_acc,
                "Testing Loss":test_loss,
                "Testing Accuracy": test_acc,
                "Prediction Samples":samples
                })
    
    if early_stop[0] :
        break
        


Testing Accuracy,41.95
Testing Loss,0.0169
Training Accuracy,42.86
Training Loss,0.01284


Testing Accuracy,▁▄▅▅▅▆▇▇▆▆▇█▇▆▇▇▇▇▇▇█▇▇▇▇█▇▇▇▇█▇▇██▇▇▇
Testing Loss,█▅▄▄▄▂▂▂▂▂▁▁▂▂▁▁▂▁▁▂▁▂▁▂▁▁▁▁▁▁▁▂▂▁▁▂▂▂
Training Accuracy,▁▆▆▇▇▇▇▇█▇████████████████████████████
Training Loss,█▄▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁



Epoch: 0
Training Loss :  0.016109103004932405 , and Accuracy =  33.72
Testing Loss :  0.021608196115493774  and Test Accuracy =  33.94

Epoch: 1
Training Loss :  0.015605215196609496 , and Accuracy =  35.454
Testing Loss :  0.022647169935703277  and Test Accuracy =  34.51

Epoch: 2
Training Loss :  0.015947474386692046 , and Accuracy =  35.328
Testing Loss :  0.020820629739761353  and Test Accuracy =  35.92

Epoch: 3
Training Loss :  0.015801969418525696 , and Accuracy =  35.434
Testing Loss :  0.02124437971115112  and Test Accuracy =  36.51

Epoch: 4
Training Loss :  0.015921190831661224 , and Accuracy =  35.408
Testing Loss :  0.02184988615512848  and Test Accuracy =  34.4

Epoch: 5
Training Loss :  0.01604870466709137 , and Accuracy =  35.148
Testing Loss :  0.02198470537662506  and Test Accuracy =  35.18

Epoch: 6
Training Loss :  0.016165034775733948 , and Accuracy =  35.11
Testing Loss :  0.021193708646297455  and Test Accuracy =  35.17

Epoch: 7
Training Loss :  0.015871588208

In [ ]:
resnet18_Freezed_Adam = wandb.Artifact('resnet18_Freezed_Adam', type='model')
resnet18_Freezed_Adam.add_file('resnet18_Freezed_Adam.pth')
run.log_artifact(resnet18_Freezed_Adam)

best_resnet18_Freezed_Adam = torch.load('resnet18_Freezed_Adam.pth')

wandb.log({
    "resnet18_Freezed_Adam_Confusion_Matrix": wandb.plot.confusion_matrix(
    preds = best_resnet18_Freezed_Adam['Predictions'],
    y_true = best_resnet18_Freezed_Adam['Target List'],
    class_names = classes) 
})

config.best_test_accuracy = best_resnet18_Freezed_Adam['Testing Accuracy']

print("Best Model accuracy for Resnet-18 Frozen with Adam Optimization" , best_resnet18_Freezed_Adam['Testing Accuracy'])

Best Model accuracy for Resnet-18 Frozen with Adam Optimization 35.92


Link to the final report is [here](https://wandb.ai/altafahmad/Asg1/reports/Scalable-Data-Mining--VmlldzoxMDM3MTg3). 